In [1]:
import numpy as np
from transformers import AutoModelForCausalLM
import openvino as ov
from nncf import compress_weights, CompressWeightsMode
import nncf
import torch

#model_id = 'stabilityai/japanese-stablelm-base-alpha-7b'
model_id = 'stabilityai/japanese-stablelm-base-gamma-7b'
model_vendor, model_name = model_id.split('/') 

"""
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = N}one,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, CausalLMOutputWithPast]:
"""

n:\work\japanese_stablelm_base\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


'\n        input_ids: Optional[torch.LongTensor] = None,\n        attention_mask: Optional[torch.FloatTensor] = None,\n        position_ids: Optional[torch.LongTensor] = None,\n        inputs_embeds: Optional[torch.FloatTensor] = None,\n        head_mask: Optional[torch.FloatTensor] = None,\n        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,\n        labels: Optional[torch.LongTensor] = None,\n        use_cache: Optional[bool] = None,\n        output_attentions: Optional[bool] = N}one,\n        output_hidden_states: Optional[bool] = None,\n        return_dict: Optional[bool] = None,\n    ) -> Union[Tuple, CausalLMOutputWithPast]:\n'

In [2]:
model = AutoModelForCausalLM.from_pretrained(f'{model_vendor}/{model_name}', trust_remote_code=True)
#model.eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.82s/it]


In [3]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [4]:
num_seq = 10
# from config.json
vocab_size = 32000
hidden_size = 4096
num_hidden_layers = 8   # alpha=32, gamma=8
num_attention_heads = 32

#past_kv = torch.Tensor(size=(1, num_attention_heads, 1, hidden_size // num_hidden_layers))
past_kv = torch.Tensor(size=(1, num_hidden_layers, 0, 128))
past_key_values = tuple([(past_kv, past_kv) for _ in range(num_attention_heads)])
#past_key_values = torch.Tensor(size=(32,2,1,num_hidden_layers,1,128))
print(len(past_key_values))
print(len(past_key_values[0]))
print(past_key_values[0][0].shape)


import random
n = 10
example_input = {
    'input_ids'     : torch.tensor([[ random.randint(5, 1000)  for _ in range(n)]], dtype=torch.int),
    'attention_mask': torch.tensor([[ 1]*n], dtype=torch.int),
    'position_ids'  : torch.tensor([[ nn for nn in range(0, n)]], dtype=torch.int),
    #'inputs_embeds': np.array([0], dtype=np.int32),
    # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
    #'head_mask': torch.Tensor(size=(num_attention_heads,1,1,1,n)).to(torch.float32),

    'past_key_values' : past_key_values,
    #'past_key_values': torch.Tensor(size=(num_hidden_layers, 2, 1, num_attention_heads, num_seq, hidden_size // num_hidden_layers)).to(torch.float32),
    #'past_key_values': torch.Tensor(size=(1, 2, 1, num_attention_heads, num_seq, hidden_size // num_hidden_layers)).to(torch.float32),
    #  past_key_values [n][0|1][ 1, 32, seq_len, 128]       # alpha
    #  past_key_values [n][0|1][ 1, 8, seq_len, 128]       # gamma

    #'labels': np.zeros((1,100), dtype=np.int32),
    'use_cache'           : torch.tensor( True, dtype=torch.bool),
    'output_attentions'   : torch.tensor(False, dtype=torch.bool),
    'output_hidden_states': torch.tensor(False, dtype=torch.bool),
    'return_dict'         : torch.tensor(False, dtype=torch.bool),
}

print(example_input['input_ids'], example_input['attention_mask'], example_input['position_ids'])
#print(example_input)

32
2
torch.Size([1, 8, 0, 128])
tensor([[808, 758,  50, 368, 847,  62, 847, 961, 659, 915]], dtype=torch.int32) tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=torch.int32) tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], dtype=torch.int32)


In [8]:
ov_model = ov.convert_model(model, example_input=example_input)
print(ov_model)

<Model: 'Model4'
inputs[
<ConstOutput: names[input_ids] shape[?,?] type: i32>,
<ConstOutput: names[attention_mask] shape[?,?] type: i32>,
<ConstOutput: names[position_ids] shape[?,?] type: i32>,
<ConstOutput: names[42, key_states.1] shape[?,8,?,128] type: f32>,
<ConstOutput: names[43] shape[?,8,?,128] type: f32>,
<ConstOutput: names[44] shape[?,8,?,128] type: f32>,
<ConstOutput: names[45] shape[?,8,?,128] type: f32>,
<ConstOutput: names[46] shape[?,8,?,128] type: f32>,
<ConstOutput: names[47] shape[?,8,?,128] type: f32>,
<ConstOutput: names[48] shape[?,8,?,128] type: f32>,
<ConstOutput: names[49] shape[?,8,?,128] type: f32>,
<ConstOutput: names[50] shape[?,8,?,128] type: f32>,
<ConstOutput: names[51] shape[?,8,?,128] type: f32>,
<ConstOutput: names[52] shape[?,8,?,128] type: f32>,
<ConstOutput: names[53] shape[?,8,?,128] type: f32>,
<ConstOutput: names[54] shape[?,8,?,128] type: f32>,
<ConstOutput: names[55] shape[?,8,?,128] type: f32>,
<ConstOutput: names[56] shape[?,8,?,128] type: f3

In [6]:
compressed_model = compress_weights(ov_model, mode=CompressWeightsMode.INT8)
ov.save_model(compressed_model, 'openvino_model_int8.xml')

n:\work\japanese_stablelm_base\venv\lib\site-packages\nncf\quantization\quantize_model.py:292: FutureWarning: `CompressWeightsMode.INT8` is deprecated.Please, use `CompressWeightsMode.INT8_ASYM` as value instead.
  warning_deprecated(


INFO:nncf:Statistics of the bitwidth distribution:
+--------------+---------------------------+-----------------------------------+
| Num bits (N) | % all parameters (layers) |    % ratio-defining parameters    |
|              |                           |             (layers)              |
+==============+===========================+===================================+
| 8            | 100% (226 / 226)          | 100% (226 / 226)                  |
+--------------+---------------------------+-----------------------------------+


n:\work\japanese_stablelm_base\venv\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [9]:
compressed_model = compress_weights(ov_model, mode=CompressWeightsMode.INT4_ASYM)
ov.save_model(compressed_model, 'openvino_model_int4asym.xml')

INFO:nncf:Statistics of the bitwidth distribution:
+--------------+---------------------------+-----------------------------------+
| Num bits (N) | % all parameters (layers) |    % ratio-defining parameters    |
|              |                           |             (layers)              |
+==============+===========================+===================================+
| 8            | 4% (2 / 226)              | 0% (0 / 224)                      |
+--------------+---------------------------+-----------------------------------+
| 4            | 96% (224 / 226)           | 100% (224 / 224)                  |
+--------------+---------------------------+-----------------------------------+
